<a href="https://colab.research.google.com/github/codingle2/Data_Analysis_Programming_Class/blob/main/crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import files
uploaded = files.upload()

Saving stockCodeInfo.xlsx to stockCodeInfo.xlsx


In [44]:
# ==========================================
# 0. 필요 라이브러리
# ==========================================

# Colab 기준 대부분 설치되어 있지만, 필요시 수동 설치
# !pip install requests pandas beautifulsoup4 openpyxl

import os
import time
import requests
import json
import re
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime


# ==========================================
# 1. 공통 유틸 함수 / 상수
# ==========================================

def parse_krw_hangeul(value):
    """
    거래대금(억) 단위의 한글 표현을 정수(원 단위)로 변환.
    예: "1,234억원" -> 123400000000
    """
    if value is None:
        return 0

    value = str(value).replace(',', '').replace(' ', '')

    if value.endswith('억원'):
        try:
            num = float(value.replace('억원', ''))
            return int(num * 100000000)
        except ValueError:
            return 0

    try:
        return int(value)
    except ValueError:
        return 0


# 네이버 m.stock 공통 헤더 (차단 방지용)
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    ),
    "Referer": "https://m.stock.naver.com/",
    "Origin": "https://m.stock.naver.com",
    "Accept": "application/json, text/plain, */*",
}


def clean_value(value):
    """
    API 값에서 '배', '원', '억원', ',' 등을 제거해
    숫자만 남기거나 깔끔한 문자열로 정리.
    """
    if not value:
        return "N/A"
    return (
        str(value)
        .replace('배', '')
        .replace('원', '')
        .replace('억원', '')
        .replace(',', '')
        .strip()
    )


# ==========================================
# 2. 거래량 상위 종목 / KOSPI 지수
# ==========================================

def get_top_stocks_data():
    """
    네이버 금융 API에서 거래량 상위 주식 데이터를 가져와
    pandas DataFrame으로 반환한다.
    (DB 저장 X, 단순 분석/출력용)
    """
    stocks_per_page = 20
    base_api_url = "https://m.stock.naver.com/api/stocks/quantTop/all"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        ),
        "Accept": "application/json, text/plain, */*",
        "Referer": "https://m.stock.naver.com/",
    }

    total_api_pages_to_fetch = 2  # 테스트용: 2페이지(40개). 필요시 늘리면 됨.
    fetched_data = []

    print(f"=== 상위 주식 데이터 크롤링 시작 (총 {total_api_pages_to_fetch * stocks_per_page}개 목표) ===")

    for i in range(1, total_api_pages_to_fetch + 1):
        full_url = f"{base_api_url}?page={i}&pageSize={stocks_per_page}"
        print(f" -> 페이지 {i} 요청 중...")

        try:
            response = requests.get(full_url, headers=headers, timeout=5)
            response.raise_for_status()
            data = response.json()

            if isinstance(data, dict) and 'stocks' in data and isinstance(data['stocks'], list):
                fetched_data.extend(data['stocks'])
            else:
                print(f" -> 페이지 {i} 데이터 없음 혹은 형식 오류.")
                break

        except Exception as e:
            print(f" -> 페이지 {i} 오류 발생: {e}")
            break

        time.sleep(0.1)  # 서버 부하 방지용 딜레이

    if not fetched_data:
        return pd.DataFrame()

    # DataFrame으로 변환 후 컬럼 정리
    df = pd.DataFrame(fetched_data)
    cols = {
        'itemCode': '종목코드',
        'stockName': '종목명',
        'closePrice': '현재가',
        'fluctuationsRatio': '등락률',
        'accumulatedTradingVolume': '거래량',
        'accumulatedTradingValueKrwHangeul': '거래대금',
    }

    available_cols = [c for c in cols.keys() if c in df.columns]
    df = df[available_cols].rename(columns=cols)

    return df


def get_kospi_info():
    """
    네이버에서 KOSPI 기본 정보 및 실시간 정보를 가져온다.
    반환값: {'basic': ..., 'realtime': ...}
    """
    basic_url = "https://m.stock.naver.com/api/index/KOSPI/basic"
    realtime_url = "https://polling.finance.naver.com/api/realtime/domestic/index/KOSPI"

    result = {}

    # 기본 정보
    try:
        res = requests.get(basic_url, timeout=5).json()
        result['basic'] = res
    except Exception as e:
        print(f"KOSPI 기본 정보 오류: {e}")
        result['basic'] = None

    # 실시간 정보
    try:
        res = requests.get(realtime_url, timeout=5).json()
        result['realtime'] = res.get('result', res)
    except Exception as e:
        print(f"KOSPI 실시간 정보 오류: {e}")
        result['realtime'] = None

    return result


def print_kospi_formatted(kospi_data):
    """
    KOSPI 정보를 콘솔용 박스 형태로 예쁘게 출력.
    """
    basic = kospi_data.get('basic')

    if not basic:
        print("KOSPI 정보를 불러오지 못했습니다.")
        return

    # 1) 기본 시세 정보
    name = basic.get('stockName', 'KOSPI')
    price = basic.get('closePrice', 'N/A')
    change = basic.get('compareToPreviousClosePrice', '0')
    ratio = basic.get('fluctuationsRatio', '0')

    # 등락 방향
    direction_info = basic.get('compareToPreviousPrice', {})
    direction_text = direction_info.get('text', '')  # 상승 / 하락 / 보합 등
    symbol = " "
    if direction_text == '상승':
        symbol = "▲"
    elif direction_text == '하락':
        symbol = "▼"

    # 거래소 정보
    exchange_info = basic.get('stockExchangeType', {})
    nation = exchange_info.get('nationName', 'KR')
    ex_name = exchange_info.get('nameKor', '코스피')

    # 시간 포맷 정리
    time_str = basic.get('localTradedAt', '')
    if 'T' in time_str:
        time_str = time_str.split('+')[0].replace('T', ' ')

    # 일봉 차트 이미지 URL
    charts = basic.get('imageCharts', {})
    day_chart = charts.get('day', '정보 없음')

    # 출력
    print("┌──────────────────────────────────────────────────┐")
    print(f"│  🇰🇷  [{nation}] {ex_name} ({name}) 상세 현황      │")
    print("├──────────────────────────────────────────────────┤")
    print(f"│  현재지수  : {price:<33} │")
    print(f"│  등락률    : {symbol} {change} ({ratio}%) {direction_text:<15} │")
    print("├──────────────────────────────────────────────────┤")
    print(f"│  장 상태   : {basic.get('marketStatus')}                          │")
    print(f"│  기준시간  : {time_str:<26}  │")
    print(f"│  거래소    : {exchange_info.get('nameEng')} ({exchange_info.get('code')})              │")
    print("├──────────────────────────────────────────────────┤")
    print("│  📊 일봉 차트 (클릭하여 보기):                   │")
    print(f"│  {day_chart:<46}  │")
    print("└──────────────────────────────────────────────────┘")


# ==========================================
# 3. 종목코드 / 일별 시세 / 재무 / 통합 정보
# ==========================================

def get_code_from_excel(stock_name, file_name='stockCodeInfo.xlsx'):
    """
    로컬 엑셀 파일에서 '회사명'으로 종목코드를 찾는다.
    - 종목코드는 0으로 시작하므로 문자열(str)로 읽어야 한다.
    """
    if not os.path.exists(file_name):
        print(f"오류: '{file_name}' 파일이 현재 폴더에 없습니다.")
        return None

    try:
        df = pd.read_excel(file_name, dtype={'종목코드': str})
        found_row = df[df['회사명'] == stock_name]

        if not found_row.empty:
            return found_row.iloc[0]['종목코드']

        print(f"'{stock_name}'을(를) 엑셀 파일에서 찾을 수 없습니다.")
        return None

    except Exception as e:
        print(f"엑셀 파일 읽기 오류: {e}")
        return None


def get_daily_stock_prices(code, pages=1):
    """
    종목코드(code)를 받아 일별 시세를 가져와 DataFrame으로 반환.
    호출 URL: https://m.stock.naver.com/api/stock/{code}/price
    """
    url = f"https://m.stock.naver.com/api/stock/{code}/price"

    all_prices = []

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://m.stock.naver.com/",
    }

    print(f"=== [{code}] 시세 데이터 요청 중... ===")

    for page in range(1, pages + 1):
        params = {"pageSize": 20, "page": page}

        try:
            response = requests.get(url, params=params, headers=headers, timeout=5)
            data = response.json()

            if len(data) > 0:
                all_prices.extend(data)
                print(f" -> {page}페이지 수집 성공 ({len(data)}건)")
            else:
                break

        except Exception as e:
            print(f"API 요청 오류: {e}")
            break

        time.sleep(0.1)  # 차단 방지용 딜레이

    if not all_prices:
        return pd.DataFrame()

    df = pd.DataFrame(all_prices)

    # 컬럼명 한글화
    rename_map = {
        'localTradedAt': '날짜',
        'closePrice': '종가',
        'compareToPreviousClosePrice': '전일비',
        'openPrice': '시가',
        'highPrice': '고가',
        'lowPrice': '저가',
        'accumulatedTradingVolume': '거래량',
        'fluctuationsRatio': '등락률',  # 있으면 함께 사용
    }

    available = [c for c in rename_map.keys() if c in df.columns]
    return df[available].rename(columns=rename_map)


def get_financial_summary(code):
    """
    연간 재무 요약 정보 가져오기.
    호출 URL: https://m.stock.naver.com/api/stock/{code}/finance/annual
    반환: 행=항목, 열=연도(YYYYMM) 인 DataFrame
    """
    url = f"https://m.stock.naver.com/api/stock/{code}/finance/annual"

    try:
        data = requests.get(url, headers=HEADERS, timeout=5).json()
        finance = data.get("financeInfo", {})
    except Exception:
        return pd.DataFrame()

    titles = finance.get("trTitleList", [])
    years = [t["key"] for t in titles]  # 예: ['202212','202312',...]

    rows = finance.get("rowList", [])
    result = {}

    for row in rows:
        title = row["title"]
        cols = row["columns"]

        row_data = {}
        for y in years:
            row_data[y] = cols.get(y, {}).get("value")

        result[title] = row_data

    df = pd.DataFrame(result).T
    df.columns = years

    return df


def get_stock_overview(code):
    """
    통합(integration) 엔드포인트를 활용해
    재무/수급/리포트/산업비교 등 다양한 정보를 한 번에 가져온다.

    호출 URL: https://m.stock.naver.com/api/stock/{code}/integration

    반환:
        {
            "totalInfos": {...},
            "dealTrend": DataFrame,
            "researchReports": list,
            "industryCompare": DataFrame,
            "consensus": {...},
            "corporationSummary": {...}
        }
    """
    url = f"https://m.stock.naver.com/api/stock/{code}/integration"

    try:
        res = requests.get(url, headers=HEADERS, timeout=5)
        data = res.json()
    except Exception:
        return {}

    result = {}

    # ① 핵심 기업지표 (시총, PER, 52주 최고/최저 등)
    total_infos = data.get("totalInfos", [])
    total_info_dict = {}
    for item in total_infos:
        key = item.get("key")
        value = item.get("value")
        total_info_dict[key] = value
    result["totalInfos"] = total_info_dict

    # ② 최근 5일 수급
    deals = data.get("dealTrendInfos", [])
    deal_list = []
    for d in deals:
        deal_list.append({
            "bizdate": d.get("bizdate"),
            "closePrice": d.get("closePrice"),
            "compareToPreviousClosePrice": d.get("compareToPreviousClosePrice"),
            "foreigner": d.get("foreignerPureBuyQuant"),
            "institution": d.get("organPureBuyQuant"),
            "individual": d.get("individualPureBuyQuant"),
            "volume": d.get("accumulatedTradingVolume"),
        })
    result["dealTrend"] = pd.DataFrame(deal_list)

    # ③ 증권사 리포트
    result["researchReports"] = data.get("researches", [])

    # ④ 산업 내 경쟁사 비교
    industry_list = []
    industry_raw = data.get("industryCompareInfo", [])
    for info in industry_raw:
        industry_list.append({
            "종목명": info.get("stockName"),
            "종가": info.get("closePrice"),
            "등락": info.get("compareToPreviousClosePrice"),
            "등락률": info.get("fluctuationsRatio"),
            "시총": info.get("marketValue"),
        })
    result["industryCompare"] = pd.DataFrame(industry_list)

    # ⑤ 목표가 컨센서스
    result["consensus"] = data.get("consensusInfo", {})

    # ⑥ 회사 개요
    result["corporationSummary"] = data.get("corporationSummary", {})

    return result


def print_comprehensive_report(stock_name, stock_code):
    """
    한 종목에 대해:
    - 핵심 지표
    - 최근 수급
    - 증권사 리포트
    - 목표주가 컨센서스
    - 경쟁사 비교
    - 연간 재무제표
    를 콘솔에 종합 출력.
    """
    if not stock_code:
        return

    print("\n" + "=" * 60)
    print(f" 📢 [{stock_name}] ({stock_code}) 종합 분석 리포트")
    print("=" * 60 + "\n")

    overview = get_stock_overview(stock_code)
    financial = get_financial_summary(stock_code)

    # 1) 핵심지표 요약
    info = overview.get("totalInfos", {})
    important_keys = [
        ("시가총액", "시총"),
        ("52주 최고", "52주 최고"),
        ("52주 최저", "52주 최저"),
        ("PER", "PER"),
        ("EPS", "EPS"),
        ("추정PER", "추정PER"),
        ("추정EPS", "추정EPS"),
        ("PBR", "PBR"),
        ("BPS", "BPS"),
        ("배당수익률", "배당수익률"),
        ("주당배당금", "주당배당금"),
    ]

    print("[1] 핵심 지표 요약")
    print("────────────────────────────────────────")
    for label, key in important_keys:
        value = info.get(key)
        if value not in ["", None, "-"]:
            print(f"{label:<12} : {value}")
    print("────────────────────────────────────────\n")

    # 2) 최근 수급
    print("[2] 최근 5일 투자자 순매수")
    print("────────────────────────────────────────")
    deal_df = overview.get("dealTrend")
    if isinstance(deal_df, pd.DataFrame) and not deal_df.empty:
        print(deal_df)
    else:
        print("데이터 없음")
    print("────────────────────────────────────────\n")

    # 3) 증권사 리포트
    print("[3] 최신 증권사 리포트 (최근 5개)")
    print("────────────────────────────────────────")
    reports = overview.get("researchReports", [])
    if reports:
        for r in reports[:5]:
            title = r.get("tit", "")
            broker = r.get("bnm", "")
            date = r.get("wdt", "")
            print(f"- {broker} : \"{title}\" ({date})")
    else:
        print("데이터 없음")
    print("────────────────────────────────────────\n")

    # 4) 목표주가 컨센서스
    print("[4] 목표주가 컨센서스")
    print("────────────────────────────────────────")
    consensus = overview.get("consensus", {})
    if consensus:
        print(f"평균 목표가 : {consensus.get('priceTargetMean')}원")
        print(f"평균 의견   : {consensus.get('recommMean')}")
        print(f"작성일     : {consensus.get('createDate')}")
    else:
        print("데이터 없음")
    print("────────────────────────────────────────\n")

    # 5) 경쟁사 비교
    print("[5] 경쟁사 비교")
    print("────────────────────────────────────────")
    ind_df = overview.get("industryCompare")
    if isinstance(ind_df, pd.DataFrame) and not ind_df.empty:
        print(ind_df)
    else:
        print("데이터 없음")
    print("────────────────────────────────────────\n")

    # 6) 연간 재무제표
    print("[6] 연간 재무제표")
    print("────────────────────────────────────────")
    if isinstance(financial, pd.DataFrame) and not financial.empty:
        print(financial)
    else:
        print("데이터 없음")
    print("────────────────────────────────────────")

    print("\n" + "=" * 60 + "\n")


# ==========================================
# 4. 실행 예시 (직접 실행 시에만 동작)
# ==========================================

if __name__ == "__main__":
    # 1) 거래량 상위 종목 목록
    print("\n[1] 거래량 상위 주식 목록")
    df_top = get_top_stocks_data()
    if not df_top.empty:
        print(df_top[['종목명', '현재가', '등락률', '거래량']].head(5))
    else:
        print("데이터를 가져오지 못했습니다.")

    # 2) KOSPI 정보
    print("\n[2] KOSPI 정보")
    kospi_data = get_kospi_info()
    print_kospi_formatted(kospi_data)

    # 3) 특정 종목 시세 + 종합 리포트
    target_name = "삼성전자"
    stock_code = get_code_from_excel(target_name)

    if stock_code:
        print(f"\n[3] '{target_name}' 시세 조회 (코드: {stock_code})")
        df_result = get_daily_stock_prices(stock_code, pages=2)

        if not df_result.empty:
            print("\n[최신 데이터 5건]")
            if '등락률' in df_result.columns:
                print(df_result[['날짜', '종가', '전일비', '등락률', '거래량']].head(5))
            else:
                print(df_result.head(5))

            # 종합 리포트 출력
            print_comprehensive_report(target_name, stock_code)
        else:
            print("시세 데이터를 가져오지 못했습니다.")
    else:
        print("\n[3] 종목 코드를 확보하지 못해 종료합니다.")



[1] 거래량 상위 주식 목록
=== 상위 주식 데이터 크롤링 시작 (총 40개 목표) ===
 -> 페이지 1 요청 중...
 -> 페이지 2 요청 중...
                종목명     현재가    등락률          거래량
0  KODEX 200선물인버스2X     678  -3.97  744,006,850
1         엔에이치스팩32호   3,840  92.00   60,567,827
2                동양     881  10.68   54,772,669
3         KODEX 인버스   2,550  -1.92   47,864,617
4  KODEX 코스닥150레버리지  11,975  -3.31   35,284,743

[2] KOSPI 정보
┌──────────────────────────────────────────────────┐
│  🇰🇷  [대한민국] 코스피 (코스피) 상세 현황      │
├──────────────────────────────────────────────────┤
│  현재지수  : 4,100.05                          │
│  등락률    : ▲ 71.54 (1.78%) 상승              │
├──────────────────────────────────────────────────┤
│  장 상태   : CLOSE                          │
│  기준시간  : 2025-12-05 19:00:48         │
│  거래소    : KOSPI (KS)              │
├──────────────────────────────────────────────────┤
│  📊 일봉 차트 (클릭하여 보기):                   │
│  https://ssl.pstatic.net/imgfinance/chart/mobile/day/KOSPI_end.png?1764928848000  │
└─────────────